In [1]:

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
import sentence_transformers

print(sentence_transformers.__version__)

2.7.0


In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import time

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Cleaned_subtitles.csv')

In [6]:
# prompt: from the above csv file take 5001 rows and convert  it as another csv file and name it

data_subset = data.head(5001)
data_subset.to_csv('/content/drive/MyDrive/data_subset.csv', index=False)


In [7]:
data = pd.read_csv('/content/drive/MyDrive/data_subset.csv')

In [11]:
data.shape

(5001, 3)

In [13]:
df = data

In [14]:
df.shape

(5001, 3)

In [15]:
df.head()

,num,name,content_clean
0,9180533,the.message.(1976).eng.1cd,in the name of god the most gracious the...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah there s princess dawn and terry with the...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumi s cells 2 episode extremely polite y...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,yumi s cells 2 episode 39 laptop firs...
4,9180600,broker.(2022).eng.1cd,if you re going to throw it away then don...


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num            5001 non-null   int64 
 1   name           5001 non-null   object
 2   content_clean  5001 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.3+ KB


In [18]:
import re

def preprocess_text(text):
    # Replace every two or more spaces with a single space
    cleaned_text = re.sub(r'\s{2,}', ' ', text)
    return cleaned_text
df['content_clean'] = df['content_clean'].apply(preprocess_text)

In [19]:
import tensorflow as tf

# Check GPU availability
if tf.test.gpu_device_name():
    print('GPU available')
else:
    print('GPU not available')

# Check TPU availability
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("TPU available")
except ValueError:
    print("TPU not available")

GPU not available
TPU not available


In [21]:
!pip install semantic_text_splitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 31.6 MB/s eta 0:00:00


In [22]:
import sys

print(sys.path)

import sys

sys.path.append("/path/to/semantic_text_splitter")

import semantic_text_splitter

print(semantic_text_splitter)

import semantic_text_splitter as text_splitter

print(text_splitter)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']
<module 'semantic_text_splitter' from '/usr/local/lib/python3.10/dist-packages/semantic_text_splitter/__init__.py'>
<module 'semantic_text_splitter' from '/usr/local/lib/python3.10/dist-packages/semantic_text_splitter/__init__.py'>


In [23]:
!pip uninstall semantic_text_splitter
!pip install semantic_text_splitter

Found existing installation: semantic-text-splitter 0.12.0
Uninstalling semantic-text-splitter-0.12.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/semantic_text_splitter-0.12.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/semantic_text_splitter/*
Proceed (Y/n)? Y
  Successfully uninstalled semantic-text-splitter-0.12.0
  Using cached semantic_text_splitter-0.12.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)


In [27]:
from semantic_text_splitter import TextSplitter
from tokenizers import Tokenizer
MAX_TOKENS = 1500
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
# Specify a capacity for the TextSplitter
splitter = TextSplitter.from_huggingface_tokenizer(tokenizer, capacity=5001)
counter = 0
def chunk_document(content):
    global counter  # Use the global counter variable
    counter += 1  # Increment the counter
    start_time = time.time()
    # Perform your chunking operation
    chunks_with_model = splitter.chunks(content,MAX_TOKENS)
    end_time = time.time()
    execution_time = end_time - start_time
    print(counter)
    print("Chunking operation completed in {:.2f} seconds ".format(execution_time))
    return chunks_with_model

 # Add the missing 'capacity' argument
#splitter = TextSplitter.from_huggingface_tokenizer(tokenizer, capacity=5001)

In [28]:
with tf.device('/device:GPU:0'):  # Or '/device:TPU:0' for TPU
# Apply chunking to each document in the DataFrame
    df['chunks'] = df['content_clean'].apply(chunk_document)

TypeError: TextSplitter.chunks() takes 1 positional arguments but 2 were given

In [29]:
def chunk_document(content):
    global counter  # Use the global counter variable
    counter += 1  # Increment the counter
    start_time = time.time()
    # Perform your chunking operation
    chunks_with_model = splitter.chunks(content)
    end_time = time.time()
    execution_time = end_time - start_time
    print(counter)
    print("Chunking operation completed in {:.2f} seconds ".format(execution_time))
    return chunks_with_model

In [30]:
with tf.device('/device:GPU:0'):  # Or '/device:TPU:0' for TPU
    # Apply chunking to each document in the DataFrame
    df['chunks'] = df['content_clean'].apply(chunk_document)

Streaming output truncated to the last 5000 lines.
2503
Chunking operation completed in 0.02 seconds 
2504
Chunking operation completed in 0.02 seconds 
2505
Chunking operation completed in 0.39 seconds 
2506
Chunking operation completed in 0.19 seconds 
2507
Chunking operation completed in 0.02 seconds 
2508
Chunking operation completed in 0.19 seconds 
2509
Chunking operation completed in 0.02 seconds 
2510
Chunking operation completed in 0.19 seconds 
2511
Chunking operation completed in 0.20 seconds 
2512
Chunking operation completed in 0.03 seconds 
2513
Chunking operation completed in 0.20 seconds 
2514
Chunking operation completed in 0.23 seconds 
2515
Chunking operation completed in 0.23 seconds 
2516
Chunking operation completed in 0.41 seconds 
2517
Chunking operation completed in 0.91 seconds 
2518
Chunking operation completed in 0.73 seconds 
2519
Chunking operation completed in 0.65 seconds 
2520
Chunking operation completed in 0.38 seconds 
2521
Chunking operation complet

In [31]:
for i, chunk in enumerate(df['chunks'][0]):
    print(f"CHUNK {i+1}: ", chunk )

CHUNK 1:  in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patriarch of alexandria kisra emperor of persia muhammad calls you with the call of god accept islam for your salvation embrace islam you come out of the desert smelling of camel and goat to tell persia where he should kneel muhammad messenger of god who gave him this authority god sent muhammad as a mercy to mankind the scholars and historians of islam the university of al azhar in cairo the high islamic congress of the shiat in lebanon the makers of this film honour the islamic tradition which holds that the impersonation of the prophet offe

In [32]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [33]:
counter = 0
def embedding(content):
    global counter  # Use the global counter variable
    counter += 1  # Increment the counter
    start_time = time.time()
    # Perform your embedding operation
    query_embedding = model.encode(content)

    end_time = time.time()
    execution_time = end_time - start_time
    print(counter)

    print("Embedding operation completed in {:.2f} seconds".format(execution_time))
    return query_embedding


In [34]:
import tensorflow as tf

with tf.device('/device:GPU:0'):  # Or '/device:TPU:0' for TPU
    df['emb'] = df['chunks'].apply(embedding)

Streaming output truncated to the last 5000 lines.
2502
Embedding operation completed in 0.15 seconds
2503
Embedding operation completed in 0.15 seconds
2504
Embedding operation completed in 0.41 seconds
2505
Embedding operation completed in 0.26 seconds
2506
Embedding operation completed in 0.16 seconds
2507
Embedding operation completed in 0.26 seconds
2508
Embedding operation completed in 0.15 seconds
2509
Embedding operation completed in 0.27 seconds
2510
Embedding operation completed in 0.27 seconds
2511
Embedding operation completed in 0.15 seconds
2512
Embedding operation completed in 0.27 seconds
2513
Embedding operation completed in 0.28 seconds
2514
Embedding operation completed in 0.27 seconds
2515
Embedding operation completed in 0.41 seconds
2516
Embedding operation completed in 0.53 seconds
2517
Embedding operation completed in 0.42 seconds
2518
Embedding operation completed in 0.39 seconds
2519
Embedding operation completed in 0.27 seconds
2520
Embedding operation comple

In [35]:
df.head()

,num,name,content_clean,chunks,emb
0,9180533,the.message.(1976).eng.1cd,in the name of god the most gracious the most...,[in the name of god the most gracious the most...,"[[-0.038742628, 0.14162855, -0.07002453, -0.06..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,ah there s princess dawn and terry with the b...,[ah there s princess dawn and terry with the b...,"[[-0.077799, -0.019269004, 0.047039453, -0.072..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,yumi s cells 2 episode extremely polite yumi ...,[yumi s cells 2 episode extremely polite yumi ...,"[[-0.14962058, -0.14706892, 0.05522112, -0.043..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,yumi s cells 2 episode 39 laptop first place ...,[yumi s cells 2 episode 39 laptop first place ...,"[[-0.11727129, -0.060495846, 0.06533917, -0.02..."
4,9180600,broker.(2022).eng.1cd,if you re going to throw it away then don t g...,[if you re going to throw it away then don t g...,"[[-0.065826505, 0.0031216356, 0.0174946, -0.04..."


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5001 entries, 0 to 5000
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   num            5001 non-null   int64 
 1   name           5001 non-null   object
 2   content_clean  5001 non-null   object
 3   chunks         5001 non-null   object
 4   emb            5001 non-null   object
dtypes: int64(1), object(4)
memory usage: 195.5+ KB


In [37]:
df['emb'][5].shape

(1, 384)

In [38]:
len(df['emb'][0])

2

In [39]:
len(df['emb'][0])

2

In [40]:
df.to_csv('video_subtitles1.csv',index=False)

In [41]:
exploded_df = df.explode(['chunks', 'emb'])

In [42]:
exploded_df.shape

(8616, 5)

In [43]:
exploded_df.tail(20)

,num,name,content_clean,chunks,emb
4990,9203064,reign.s01.e11.inquisition.(2014).eng.1cd,since mary queen of scotland was a child the ...,since mary queen of scotland was a child the e...,"[-0.109299906, -0.032154594, 0.019658908, 0.02..."
4990,9203064,reign.s01.e11.inquisition.(2014).eng.1cd,since mary queen of scotland was a child the ...,me i m a mother and a mother who won t be here...,"[-0.10176695, -0.023575583, -0.002915104, -0.0..."
4991,9203065,reign.s01.e12.royal.blood.(2014).eng.1cd,support us and become vip member to remove al...,support us and become vip member to remove all...,"[-0.039389413, -0.029644612, 0.010620603, -0.0..."
4991,9203065,reign.s01.e12.royal.blood.(2014).eng.1cd,support us and become vip member to remove al...,us hours to dig her a grave so don t there s n...,"[-0.022119377, -0.02855085, 0.016108641, -0.07..."
4992,9203066,reign.s01.e13.the.consummation.(2014).eng.1cd,api opensubtitles org is deprecated please im...,api opensubtitles org is deprecated please imp...,"[-0.011148849, 0.0071715596, -0.017482618, -0...."
4993,9203067,reign.s01.e14.dirty.laundry.(2014).eng.1cd,advertise your product or brand here contact ...,advertise your product or brand here contact w...,"[-0.021537833, -0.029339032, 0.039726987, -0.0..."
4994,9203068,reign.s01.e15.the.darkness.(2014).eng.1cd,support us and become vip member to remove al...,support us and become vip member to remove all...,"[-0.030421587, -0.05153956, 0.068494245, -0.01..."
4994,9203068,reign.s01.e15.the.darkness.(2014).eng.1cd,support us and become vip member to remove al...,of scots dangled herself in front of my sons a...,"[0.020256074, 0.020579405, 0.029884217, -0.029..."
4995,9203069,reign.s01.e16.monsters.(2014).eng.1cd,use the free code joinnow at www playships eu...,use the free code joinnow at www playships eu ...,"[-0.09329112, -0.035985734, 0.068635985, 0.007..."
4995,9203069,reign.s01.e16.monsters.(2014).eng.1cd,use the free code joinnow at www playships eu...,invited now on pain of death and so under the ...,"[-0.050080568, 0.007956857, 0.06591065, 0.0065..."


In [44]:
exploded_df.to_csv('full_video_subtitles.csv',index=False)

In [45]:
exploded_df = exploded_df.reset_index(drop=True)

In [46]:
exploded_df.shape

(8616, 5)

In [47]:
embeddings = exploded_df['emb'].tolist()
ids = exploded_df.index.astype(str).tolist()  # Convert integers to stringsids = cleaned_df.index.astype(str).tolist()

documents = exploded_df['chunks'].tolist()
metadata = exploded_df.drop(['content_clean','chunks','emb'], axis = 1).to_dict(orient = 'records')

In [51]:
documents[8500]

'0 12 59 75 0 13 01 53 default 0000 0000 0000 i ll never forgive you lucifer dialogue 0 0 13 19 62 0 13 25 50 default 0000 0000 0000 how did you plan to defeat me nwhen you can t recover your celestial force dialogue 0 0 13 27 54 0 13 29 91 default 0000 0000 0000 what s so funny dialogue 0 0 13 30 58 0 13 34 04 default 0000 0000 0000 i certainly don t want to nbattle alongside a i1 hero i0 dialogue 0 0 13 34 04 0 13 37 70 default 0000 0000 0000 but i ve sworn my allegiance to king satan dialogue 0 0 13 37 70 0 13 39 04 default 0000 0000 0000 alciel dialogue 0 0 13 39 29 0 13 42 30 default 0000 0000 0000 which makes you my current enemy lucifer dialogue 0 0 13 42 91 0 13 44 75 default 0000 0000 0000 you re okay after all dialogue 0 0 13 44 75 0 13 50 41 default 0000 0000 0000 his majesty mercifully gave me enough nmagic to return from death s door dialogue 0 0 13 50 41 0 13 52 56 default 0000 0000 0000 where have you been then dialogue 0 0 13 53 04 0 13 55 98 default 0000 0000 0000 i we

In [52]:
embeddings[8500]

array([ 2.01030113e-02,  2.17951238e-02, -1.84650905e-02, -6.27300441e-02,
        1.33877480e-02, -3.82660003e-03,  2.22215466e-02, -2.70831212e-02,
        5.31688742e-02, -2.28082500e-02, -8.13314542e-02, -6.12697378e-02,
        4.22885902e-02, -3.72990184e-02, -2.13730913e-02, -3.05292662e-02,
       -7.33355153e-03,  4.89875525e-02, -5.48706055e-02,  2.13599354e-02,
        5.47259934e-02,  1.17636383e-01,  3.08944229e-02,  6.67601358e-03,
       -8.06998163e-02,  2.72451248e-02, -1.04030482e-01,  1.73653588e-02,
       -6.26386330e-02, -1.23516060e-01, -2.87173805e-03,  6.94965720e-02,
        1.10959094e-02,  3.35061327e-02, -6.63727373e-02,  8.66799280e-02,
       -1.44816320e-02,  5.31831272e-02, -5.95640838e-02, -5.77999018e-02,
       -6.91386461e-02,  3.53694186e-02, -5.79645634e-02, -1.63405035e-02,
       -1.57120191e-02, -1.24909114e-02, -1.32433683e-01, -9.22487024e-03,
        9.78189707e-02, -3.97786237e-02, -1.06673524e-01,  2.64425743e-02,
        5.85755967e-02,  

In [53]:
embeddings_as_lists = [embedding.tolist() for embedding in embeddings]

In [55]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7

In [56]:
import chromadb
import chromadb.config

from chromadb.config import Settings

client = chromadb.PersistentClient(path="/content")

In [57]:
collection = client.create_collection(name="Search_Engine",
                                      metadata={"hnsw:space": "cosine"} # l2 is the default
)

In [59]:
for i, embedding in enumerate(embeddings_as_lists):

    # Add the embeddings list to your collection
    collection.add(
            documents=documents[i],
            embeddings=embeddings_as_lists[i],
            ids=ids[i],
            metadatas=metadata[i]
        )

In [60]:
results = collection.query(
    query_texts=["up when he has fallen   is described by them as upsetting social order   to this inhumanity has come a man   whom god chose   and in that we believe   you ve overcome  i beg you to collect yourself   i speak of the messenger of god   muhammad teaches us to worship one god   to speak truth   to love our neighbors as ourselves   to give charity even a smile can be charity   to protect women from misuse   to shelter orphans   and to turn away from gods of wood and stone   i cannot keep still and hear this blasphemy   we are an ancient civilization   to call our gods wood and stone is to speak ignorantly of them   the idol the form is not what we worship   but the spirit that resides within the form   i agree that idolatry is not always fully understood   thank you   now let me bring him back to the women   god made woman to be the proper companion of man   she is different but equal   equal   we buy them   feed them  clothe them   use them  discard them   women equal to us   god created man from one male and one female   amr  you must respect in all woman the womb that bore you   why are your  guards so tongue tied   while this only guard is eloquent   god has spoken to us before   through abraham  noah  moses and through jesus christ   why should we be so surprised that god speaks to us now through muhammad   who taught you those names   they are named in the quran   i knew muhammad when he was an orphan minding sheep   and we knew christ as a carpenter   what christ says and what your muhammad says   is like two raised from the same land   they are lying to you they deny christ   you worship three gods  they say father  son and holy ghost  they say   what do you say of christ   they say god cannot have a son   christ is not the son of god   speak to me of christ   we say of christ what our prophet has taught us   that god cast his holy spirit into the womb of a virgin named  mary   and that she conceived christ  the apostle of god   the apostle he says not the son  not the son   what does your miracle  your quran say of the birth of our dear lord jesus christ   may i relate the words   come closer to me   in the name of god most gracious  most merciful   relate in the book  the story of mary   how she withdrew from her family to a place in the east   how we sent to her our angel  gabriel  who said   i am a messenger from your god   to announce the birth of a holy son to you   she said   how shall i  mary  have a son when no man has touched me   and gabriel replied   for your lord says  it will happen   we appoint him as a sign onto man   and a mercy from us"],
    n_results=5
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 40.2MiB/s]


In [61]:
print(results
)

{'ids': [['0', '3167', '718', '7910', '8070']], 'distances': [[0.5292237997055054, 0.5485879778862, 0.5524751543998718, 0.5578075647354126, 0.5592724084854126]], 'metadatas': [[{'name': 'the.message.(1976).eng.1cd', 'num': 9180533}, {'name': 'love.wont.let.me.wait.(2015).eng.1cd', 'num': 9189851}, {'name': 'man.of.god.(2021).eng.1cd', 'num': 9183341}, {'name': 'heartsong.(2022).eng.1cd', 'num': 9201473}, {'name': 'grantchester.s06.e08.episode.6.8.(2021).eng.1cd', 'num': 9201906}]], 'embeddings': None, 'documents': [['in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patriarch of alexandria kisra empero

In [62]:
# Connect to the ChromaDB client and collection
client = chromadb.PersistentClient(path="/content")

collection = client.get_collection("Search_Engine")

In [63]:
model = SentenceTransformer('all-MiniLM-L6-v2')
query_embedding = model.encode(["up when he has fallen   is described by them as upsetting social order   to this inhumanity has come a man   whom god chose   and in that we believe   you ve overcome  i beg you to collect yourself   i speak of the messenger of god   muhammad teaches us to worship one god   to speak truth   to love our neighbors as ourselves   to give charity even a smile can be charity   to protect women from misuse   to shelter orphans   and to turn away from gods of wood and stone   i cannot keep still and hear this blasphemy   we are an ancient civilization   to call our gods wood and stone is to speak ignorantly of them   the idol the form is not what we worship   but the spirit that resides within the form   i agree that idolatry is not always fully understood   thank you   now let me bring him back to the women   god made woman to be the proper companion of man   she is different but equal   equal   we buy them   feed them  clothe them   use them  discard them   women equal to us   god created man from one male and one female   amr  you must respect in all woman the womb that bore you   why are your  guards so tongue tied   while this only guard is eloquent   god has spoken to us before   through abraham  noah  moses and through jesus christ   why should we be so surprised that god speaks to us now through muhammad   who taught you those names   they are named in the quran   i knew muhammad when he was an orphan minding sheep   and we knew christ as a carpenter   what christ says and what your muhammad says   is like two raised from the same land   they are lying to you they deny christ   you worship three gods  they say father  son and holy ghost  they say   what do you say of christ   they say god cannot have a son   christ is not the son of god   speak to me of christ   we say of christ what our prophet has taught us   that god cast his holy spirit into the womb of a virgin named  mary   and that she conceived christ  the apostle of god   the apostle he says not the son  not the son   what does your miracle  your quran say of the birth of our dear lord jesus christ   may i relate the words   come closer to me   in the name of god most gracious  most merciful   relate in the book  the story of mary   how she withdrew from her family to a place in the east   how we sent to her our angel  gabriel  who said   i am a messenger from your god   to announce the birth of a holy son to you   she said   how shall i  mary  have a son when no man has touched me   and gabriel replied   for your lord says  it will happen   we appoint him as a sign onto man   and a mercy from us"])

In [64]:
results = collection.query(
                query_embeddings=query_embedding,
                n_results=5,
            )

In [65]:
results

{'ids': [['0', '3167', '718', '7910', '8070']],
 'distances': [[0.5292239189147949,
   0.5485879778862,
   0.5524752140045166,
   0.5578075051307678,
   0.5592725276947021]],
 'metadatas': [[{'name': 'the.message.(1976).eng.1cd', 'num': 9180533},
   {'name': 'love.wont.let.me.wait.(2015).eng.1cd', 'num': 9189851},
   {'name': 'man.of.god.(2021).eng.1cd', 'num': 9183341},
   {'name': 'heartsong.(2022).eng.1cd', 'num': 9201473},
   {'name': 'grantchester.s06.e08.episode.6.8.(2021).eng.1cd',
    'num': 9201906}]],
 'embeddings': None,
 'documents': [['in the name of god the most gracious the most merciful from muhammad the messenger of god to heraclius the emperor of byzantium greetings to him who is the follower of righteous guidance i bid you to hear the divine call i am the messenger of god to the people accept islam for your salvation he speaks of a new prophet in arabia was it like this when john the baptist came to king herod out of the desert crying about salvation to muqawqis patr

In [68]:
# prompt: create a streamlit app for this search engine project

import streamlit as st
import chromadb
import chromadb.config

from sentence_transformers import SentenceTransformer

# Set up ChromaDB client and collection
client = chromadb.PersistentClient(path="/content")
collection = client.get_collection("Search_Engine")

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Streamlit app
st.title("Search Engine")

# Get user query
query_text = st.text_area("Enter your query:")

# Submit button
if st.button("Search"):
    # Encode the query text
    query_embedding = model.encode(query_text)

    # Search the collection
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=5,
    )

    # Display the results
    for result in results:
        st.write(result.document)


2024-04-25 19:02:03.285 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


In [67]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [ ]:
from flask import Flask, render_template, request
from sentence_transformers import SentenceTransformer
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import chromadb

app = Flask(__name__)

# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Connect to the ChromaDB client and collection
client = chromadb.PersistentClient(path="/content")

collection = client.get_collection("Search_Engine")

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        query = request.form['query']

        # Preprocess and encode the query
        cleaned_query = clean_text(query)
        query_embedding = model.encode([cleaned_query])

        # Perform the query
        results = collection.query(
            query_embeddings=query_embedding,
            n_results=5,
            include=['documents', 'metadatas']  # Assuming this correctly fetches documents and metadata
        )

        documents = results.get('documents', [])
        metadatas = results.get('metadatas', [])

        # Combine documents with their metadata
        processed_documents = []
        for doc, meta in zip(documents[0], metadatas[0]):  # Assuming each is a list and they're parallel
            processed_documents.append({'document': doc, 'metadata': meta})

        return render_template('index.html', query=query, documents=processed_documents)
    return render_template('index.html')


def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())

    # Tokenize the text
    tokens = word_tokenize(clean_text)

    # Remove stopwords and lemmatize tokens
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word.lower() not in stop_words]

    # Join the filtered tokens back into a string
    clean_text = ' '.join(clean_tokens)

    return clean_text.strip()

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [ ]:
# prompt: what are the libraries is requried to run the app and how to install them in vs code

**Libraries required:**

- `streamlit`: `pip install streamlit`
- `sentence_transformers`: `pip install sentence_transformers`
- `chromadb`: `pip install chromadb`

**How to install them in VS Code:**

1. Open the VS Code terminal.
2. Run the following commands:

